In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(rc={'figure.figsize': (30, 15)}, font_scale=1.5)

df = pd.read_csv('ECH_2019.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107871 entries, 0 to 107870
Data columns (total 18 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   id_hogar                           107871 non-null  int64 
 1   sexo                               107871 non-null  object
 2   edad                               107871 non-null  int64 
 3   ascendencia                        107871 non-null  object
 4   parentesco                         107871 non-null  object
 5   estado_laboral                     107871 non-null  object
 6   sueldo                             107871 non-null  int64 
 7   busca_otro_trabajo                 107871 non-null  object
 8   actividad                          107871 non-null  object
 9   asistencia alguna vez a enseñanza  107871 non-null  object
 10  edu preescolar                     107871 non-null  object
 11  edu primaria                       107871 non-null  

In [2]:
df.sample(5)

,id_hogar,sexo,edad,ascendencia,parentesco,estado_laboral,sueldo,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof
84926,2019045758,Mujer,68,Blanca,Jefe/a de hogar,Inactivo: jubilado,0,0,.,Sí,No asistió,"Sí, asistió",No asistió,"Sí, asistió",0,0,0,0
4541,2019002398,Mujer,1,Blanca,Hijo/a solo del jefe/a,Menores de 14 años,0,0,.,Sí,0,No asistió,0,0,0,0,0,0
11840,2019006392,Hombre,66,Blanca,Jefe/a de hogar,Ocupados,0,No,Transporte de carga por carretera,Sí,No asistió,"Sí, asistió",No asistió,No asistió,0,0,0,0
5670,2019003012,Mujer,8,Blanca,Hijo/a de ambos,Menores de 14 años,0,0,.,Sí,"Sí, asistió",Asiste actualmente,0,0,0,0,0,0
62524,2019033641,Hombre,28,Blanca,Yerno/nuera,Ocupados,28000,No,Comercio al por menor de combustible para vehí...,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,0,0,0,0


In [3]:
no_data = {0: 'No hay dato', '0': 'No hay dato'}

mask = df.columns.difference(['edad', 'sueldo', 'hijos'])

df[mask] = df[mask].replace(no_data)

df.sample(5)

,id_hogar,sexo,edad,ascendencia,parentesco,estado_laboral,sueldo,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof
92753,2019049926,Mujer,3,Blanca,Hijo/a de ambos,Menores de 14 años,0,No hay dato,.,Sí,Asiste actualmente,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato
2696,2019001428,Mujer,18,Blanca,Hijo/a solo del jefe/a,Desocupados propiamente dichos,0,No hay dato,.,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,No hay dato,No hay dato,No hay dato,No hay dato
90971,2019048939,Mujer,26,Afro o negra,Yerno/nuera,Ocupados,18000,No,Comercio al por menor en almacenes no especial...,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,No hay dato,No hay dato,No hay dato,No hay dato
106622,2019057407,Hombre,0,Blanca,Nieto/a,Menores de 14 años,0,No hay dato,.,No,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato
34306,2019018458,Hombre,55,Blanca,Esposo/a o compañero/a,Inactivo: jubilado,0,No hay dato,.,Sí,No asistió,"Sí, asistió",No asistió,No asistió,No hay dato,No hay dato,No hay dato,No hay dato


In [4]:
df['estado_laboral'].value_counts()

Ocupados                                      49036
Menores de 14 años                            18064
Inactivo: jubilado                            16025
Inactivo: estudiante                           8216
Inactivo: realiza los quehaceres del hogar     5936
Inactivo: pensionista                          4438
Desocupados propiamente dichos                 3409
Inactivo: otro                                 1291
Desocupados buscan trabajo por primera vez      787
Inactivo: rentista                              414
Desocupados en seguro de paro                   255
Name: estado_laboral, dtype: int64

Seleccionamos personas con edad suficiente para trabajar

In [5]:
df = df[~df['estado_laboral'].str.contains('Menores de 14 años')]

Codificamos los valores de parentezco

In [6]:
df['parentesco_num'] = df['parentesco'].astype('category').cat.codes

df[['parentesco_num', 'parentesco']].value_counts()

parentesco_num  parentesco                             
6               Jefe/a de hogar                            42507
1               Esposo/a o compañero/a                     22574
3               Hijo/a de ambos                             9558
5               Hijo/a solo del jefe/a                      8574
10              Padre/madre                                 1148
7               Nieto/a                                     1107
4               Hijo/a solo del esposo/a compañero/a        1104
2               Hermano/a                                    920
13              Yerno/nuera                                  766
8               Otro no pariente                             583
9               Otro pariente                                556
12              Suegro/a                                     254
0               Cuñado/a                                     140
11              Servicio doméstico o familiar del mismo       16
dtype: int64

Creamos una columna que cuenta la cantidad de hijos por grupo familiar

In [7]:
df['hijos'] = (
    df.groupby('id_hogar')['id_hogar'].transform('count') - 1
)

# CATEGORIZAR NUMERICOS

In [ ]:
df['rango_sueldos'] = pd.cut(df['sueldo'], bins=[0, 10, 50, float('Inf'
                             )], labels=['xxx', 'yyy', 'zzz'])


### AJUSTANDO DATOS PARA ENTRENAR EL MODELO

Primero encodificamos las variables categóricas

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df = df.apply(le.fit_transform)

df.head(5)

### Creamos un mapa de correlación para investigar la relación entre las variables

In [ ]:
corr = df.corr()
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot=True)

### DEFINIMOS LAS VARIABLES DEPENDIENTES E INDEPENDIENTES

In [ ]:
# Feautures
X = df.drop('estado_laboral', axis=1)
y = df['estado_laboral']

print('X shape', X.shape, '\nY shape', y.shape)

In [ ]:
np.unique(y)

### Dividimos los datos en un 30% de test y 70% de entrenamiento

In [ ]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### DEFINIMOS ALGORITMO DE REFRESIÓN LOGISTICA Y VALOR DE CROSS VALIDATION PARA CADA FOLD

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_reg = LogisticRegression(max_iter=20000)

clf = log_reg.fit(X_train.astype('int'), y_train.astype('int'))

scores = cross_val_score(clf, X.astype('int'), y.astype('int'), cv=10)

scores

Promedio de scores en el Cross Validation

In [ ]:
scores.mean()

Scores de cada clasificación

In [ ]:
clf.score(X_test, y_test)

In [ ]:
clf.score(X_train, y_train)

### DEFINIMOS UNA PREDICCIÓN Y LA APLICAMOS SOBRE EL MODELO

In [ ]:
prediction = np.array([10000, 20000, 5000, 80000, 3000, 100,30000]).reshape(-1,1)
log_reg.predict(prediction)

### INVESTIGAMOS LA EFECTIVIDAD DEL MODELO SOBRE LOS DATOS

In [ ]:
from sklearn.metrics import accuracy_score

#Prediccion en Train
y_train_pred = log_reg.predict(X_train)

#Prediccion en Test
y_test_pred = log_reg.predict(y_test.reshape(-1, 1))

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy)
print('% de aciertos sobre el set de evaluación:',test_accuracy)

Determinamos la probabilidad de la predicción

In [ ]:
probabilidades_prediccion = log_reg.predict_proba(prediction)
probabilidades_prediccion[:,1]